<a href="https://colab.research.google.com/github/RifatMuhtasim/Deep_Learning/blob/main/LSTM/LSTM_Multivariate_Time_Series_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gdown
import os

# Replace 'output_path' with the path where you want to save the file
output_path = 'co2_emissions_dataset.csv'

if os.path.exists(output_path):
  print("File exists!")
else:
  print("File does not exist.")
  # Replace 'file_id' with the ID of your file in Google Drive
  file_id = '17CIlzVCBCBA4pQXFV0QEBdHsHbYzY18R'
  gdown.download(f'https://drive.google.com/uc?id={file_id}', output_path, quiet=False)

File exists!


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential

In [ ]:
df = pd.read_csv("/content/co2_emissions_dataset.csv")
df = df[df['Country Name'] == 'Bangladesh']
df.head()

,Country Code,Country Name,Indicator,2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015]
180,Ban,Bangladesh,Access to electricity (% of population),32,35.03246689,37.7541275,40.47424698,40.6,44.23,50.52510246,46.5,54.26874542,57.07751465,55.26,59.6,65.54574585,61.5,62.4,74.00247192
181,Ban,Bangladesh,Agricultural land (% of land area),72.21325958,72.23630637,71.85219329,71.66013674,71.62172544,71.5295383,71.29138818,71.18383652,71.08396712,71.03019129,70.99177998,70.12368441,70.0852731,71.27602366,71.59867865,72.47630022
182,Ban,Bangladesh,"Annual freshwater withdrawals, total (% of int...",..,..,..,..,..,..,..,..,34.16190476,34.16190476,34.16190476,34.16190476,34.16190476,34.16190476,34.16190476,34.16190476
183,Ban,Bangladesh,Arable land (% of land area),64.14688484,63.78581854,63.40170546,63.20964892,62.55665668,60.77437198,60.53622186,60.04455712,59.94468772,59.89091188,59.85250058,58.98440501,58.92294692,58.98440501,58.91526465,59.40109088
184,Ban,Bangladesh,Forest area (% of land area),14.75246293,14.72789045,14.70331797,14.67874549,14.654173,14.62960052,14.60502804,14.58045556,14.55588308,14.53130983,14.50673734,14.4991442,14.49155105,14.4839579,14.47636475,14.46877161


In [ ]:
df = df.drop(['Country Code'], axis="columns")

# Converting selected numerical columns to numeric type
def Converting_numeric_type(df):
  numerical_columns = df.columns[2:]
  df[numerical_columns] = df[numerical_columns].apply(pd.to_numeric, errors="coerce")
  return df

df = Converting_numeric_type(df)


# Fill Missing value
def fill_missing(row):
  for i in range(len(row)):

    if i == 0 and pd.isnull(row[i]):
      min_value = np.min(row[1:])
      row[i] = min_value
    elif pd.isnull(row[i]):
      row[i] = row[i-1]
  return row

df_numeric = df.iloc[:, 2:].copy()
df_cat = df.iloc[:, :2].copy()
df_numeric_ffill = df_numeric.apply(fill_missing, axis=1)
df = pd.concat([df_cat, df_numeric_ffill], axis="columns")

In [ ]:
# Row Column Conversion
def Row_column_conversion(df):

  # Iterate over unique country names in the 'Country Name' column
    country_df = df.drop(['Country Name'], axis="columns")
    country_df = country_df.set_index("Indicator")

    # Transpose the DataFrame to swap rows and columns
    country_df = country_df.transpose()

    country_df.reset_index(inplace=True)
    country_df.rename(columns={"index": "Year"}, inplace=True)
    country_df['Year'] = country_df['Year'].apply(lambda x: int(x.split()[0]))
    country_df.columns = country_df.columns.rename(None)
    return country_df

df = Row_column_conversion(df)
df

,Year,Access to electricity (% of population),Agricultural land (% of land area),"Annual freshwater withdrawals, total (% of internal resources)",Arable land (% of land area),Forest area (% of land area),Electric power consumption (kWh per capita),Energy use (kg of oil equivalent per capita),Renewable electricity output (% of total electricity output),Renewable energy consumption (% of total final energy consumption),Population growth (annual %),GDP per capita (current US$),CO2 emissions (metric tons per capita)
0,2000,32.000000,72.213260,34.161905,64.146885,14.752463,103.364472,141.353609,4.749223,60.20,1.905524,413.100185,0.160122
1,2001,35.032467,72.236306,34.161905,63.785819,14.727890,113.761259,151.715323,5.709522,55.96,1.899253,410.048541,0.194687
2,2002,37.754128,71.852193,34.161905,63.401705,14.703318,121.462808,152.480017,4.012858,54.47,1.858027,407.962968,0.203401
3,2003,40.474247,71.660137,34.161905,63.209649,14.678745,127.769893,157.890086,3.799716,52.62,1.746537,440.714405,0.209737
4,2004,40.600000,71.621725,34.161905,62.556657,14.654173,163.160493,158.163041,3.034477,52.05,1.661192,469.116458,0.219955
5,2005,44.230000,71.529538,34.161905,60.774372,14.629601,173.731815,161.758868,2.832079,50.66,1.517975,492.808649,0.232135
6,2006,50.525102,71.291388,34.161905,60.536222,14.605028,194.581978,171.227835,2.536318,48.69,1.210590,503.538332,0.251722
7,2007,46.500000,71.183837,34.161905,60.044557,14.580456,203.994932,177.020097,2.415739,47.19,1.051117,552.338935,0.263587
8,2008,54.268745,71.083967,34.161905,59.944688,14.555883,205.932668,184.156439,2.776317,45.20,0.887833,630.108979,0.285933
9,2009,57.077515,71.030191,34.161905,59.890912,14.531310,224.843005,192.043948,1.121661,43.10,0.880093,698.520945,0.305030


In [ ]:
co2_df = df[['Year', 'CO2 emissions (metric tons per capita)']]
co2_df.tail()

,Year,CO2 emissions (metric tons per capita)
11,2011,0.361557
12,2012,0.387829
13,2013,0.408788
14,2014,0.425189
15,2015,0.463517


# Data Preprocessing

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

train = df.drop(['Year'], axis="columns")

# scaler = StandardScaler()
scaler = MinMaxScaler()
scaler.fit(train)
train_scaled = scaler.transform(train)

In [ ]:
def Prepare_data(df, n_past, n_future):
    X_train = []  # List to store input sequences
    y_train = []  # List to store corresponding target values
    y_index = -1  # Index of the target variable

    # Iterate over the data to create input-output pairs
    for i in range(n_past, len(df) - n_future + 1):
        # Extract the input sequence of length 'n_past'
        input_seq = df[i - n_past:i, :df.shape[1]]
        X_train.append(input_seq)

        # Extract the corresponding target value 'n_future' steps ahead
        target_value = df[i + n_future - 1:i + n_future, y_index]
        y_train.append(target_value)

    # Convert lists to numpy arrays for further processing
    return np.array(X_train), np.array(y_train)

In [ ]:
n_future = 1
n_past = 5

X_train, y_train = Prepare_data(train_scaled, n_past, n_future)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

X_train shape: (11, 5, 12)
y_train shape: (11, 1)


In [ ]:
lstm_model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), activation="relu", return_sequences=True),
    LSTM(32, activation="relu"),
    Dense(1)
])

lstm_model.compile(optimizer="adam", loss="mse")
print(lstm_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 64)             19712     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 32161 (125.63 KB)
Trainable params: 32161 (125.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=1000, verbose=0)

# Extract loss and accuracy values of the last epoch
last_epoch_loss = history.history['loss'][-1]
print("Last Epoch Loss:", last_epoch_loss)

Last Epoch Loss: 2.1029018171248026e-05


In [ ]:
future_step = 10
forecast = lstm_model.predict(X_train[-future_step: ])

1/1 [==============================] - 0s 311ms/step


In [ ]:
forecast

array([[0.29666406],
       [0.34494308],
       [0.410906  ],
       [0.48517388],
       [0.585042  ],
       [0.6697635 ],
       [0.74850035],
       [0.81943357],
       [0.87314415],
       [0.99967194]], dtype=float32)

In [ ]:
forecast_copies = np.repeat(forecast, train_scaled.shape[1], axis=-1)
y_pred_future = scaler.inverse_transform(forecast_copies)[:, 0]

lstm_output = y_pred_future.tolist()

In [ ]:
pred_result_df = pd.DataFrame(columns=["Year", "CO2 emissions (metric tons per capita)"])

for idx, value in enumerate(lstm_output):
    new_df = pd.DataFrame({
        "Year": [idx + 2016],
        "CO2 emissions (metric tons per capita)": [value]
    })
    pred_result_df = pd.concat([pred_result_df, new_df], ignore_index=True)

co2_df = pd.concat([co2_df, pred_result_df], ignore_index=True)

In [ ]:
co2_df

,Year,CO2 emissions (metric tons per capita)
0,2000,0.160122
1,2001,0.194687
2,2002,0.203401
3,2003,0.209737
4,2004,0.219955
5,2005,0.232135
6,2006,0.251722
7,2007,0.263587
8,2008,0.285933
9,2009,0.305030
